<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/multi_GPU_computing_lambda_ai_gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import numba
from numba import cuda
import math
import time
from scipy.stats import norm # For Black-Scholes comparison
from tqdm import tqdm # Import tqdm
import sys # Import sys for exiting
import os # Import os to get environment variables
import google.generativeai as genai # Import the Gemini client library

# --- Option Parameters ---
S0 = 100.0  # Start Price
K = 100.0   # Strike Price
r = 0.02    # Risk-free Interest Rate
sigma = 0.20  # Volatility
T = 1.0     # Time to Maturity (years)
N_STEPS = 100 # Number of Time Steps
N_SIMULATIONS = 1_000_000
#N_SIMULATIONS = 1_00

# --- Numba CUDA Kernel ---
@cuda.jit
def monte_carlo_option_kernel(s0, k, r, sigma, dt, n_steps, paths_device, payoffs_device):
    """
    CUDA kernel to simulate stock paths and calculate option payoffs.
    Each thread simulates one path.
    """
    # Get the index of the current thread (which corresponds to a simulation path)
    tid = cuda.grid(1)

    # Only process if the thread index is within the number of simulations
    # We use payoffs_device.shape[0] here as the upper bound
    if tid < payoffs_device.shape[0]:
        s_t = s0
        # Use the pre-generated random numbers specific to this path
        path_random_numbers = paths_device[tid, :]

        # Simulate the stock price path using Geometric Brownian Motion
        for i in range(n_steps):
            diffusion_term = sigma * math.sqrt(dt) * path_random_numbers[i]
            drift_term = (r - 0.5 * sigma**2) * dt
            log_return = drift_term + diffusion_term
            s_t = s_t * math.exp(log_return)

        # Calculate the payoff at maturity T
        payoff = max(0.0, s_t - k)

        # Store the calculated payoff for this path
        payoffs_device[tid] = payoff

# --- CPU Monte Carlo Function (for comparison) ---
def monte_carlo_option_cpu(s0, k, r, sigma, t, n_steps, n_simulations):
    dt = t / n_steps
    total_payoff = 0.0

    # Wrap the range with tqdm for a progress bar
    for i in tqdm(range(n_simulations), desc="CPU Simulation"):
        s_t = s0
        random_numbers = np.random.standard_normal(n_steps)
        for j in range(n_steps):
            s_t = s_t * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * random_numbers[j])

        payoff = max(0.0, s_t - k)
        total_payoff += payoff

    # Discount the average payoff back to present value
    option_price = (total_payoff / n_simulations) * np.exp(-r * t)
    return option_price

# --- Black-Scholes Formula (for theoretical benchmark) ---
def black_scholes_price(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call_price

# --- Main Execution ---
print("----------------------------------------")
print("European Call Option Pricing using Monte Carlo Simulation (Corrected)")
print("----------------------------------------")

# --- Detect and print number of GPUs ---
gpu_available = cuda.detect()
if gpu_available:
    try:
        # Try the potentially newer count_devices first (though traceback suggests it's not there)
        gpu_count = cuda.count_devices()
    except AttributeError:
        # Fallback for older versions using list_devices
        print("cuda.count_devices() not found, falling back to len(list(cuda.list_devices()))")
        gpu_count = len(list(cuda.list_devices()))
    except Exception as e:
        print(f"An error occurred detecting GPUs: {e}")
        gpu_count = 0 # Assume 0 if detection fails unexpectedly

    print(f"Number of CUDA capable GPUs detected: {gpu_count}")
else:
    print("No CUDA capable GPU detected. GPU calculation will not run.")
    gpu_count = 0 # Ensure gpu_count is set even if no GPU
print("----------------------------------------")

# --- Check if GPU is available for GPU calculation block ---
run_gpu_calculation = gpu_available and gpu_count > 0

print(f"Number of Simulations: {N_SIMULATIONS:,}")
print(f"Start Price: {S0:.2f}")
print(f"Strike Price: {K:.2f}")
print(f"Expected Return (mu): {r:.2f}") # Using r as drift proxy for printing consistency
print(f"Volatility (sigma): {sigma:.2f}")
print(f"Time to Maturity (T): {T:.2f} years")
print(f"Number of Time Steps: {N_STEPS}")
print(f"Risk-free Interest Rate (r): {r:.2f}")
print("----------------------------------------")

# --- CPU Calculation ---
start_time_cpu = time.time()
option_price_cpu = monte_carlo_option_cpu(S0, K, r, sigma, T, N_STEPS, N_SIMULATIONS)
end_time_cpu = time.time()
cpu_time = end_time_cpu - start_time_cpu

print(f"Option Price (CPU): {option_price_cpu:.4f}")
print(f"Execution Time (CPU): {cpu_time:.4f} seconds")


# --- GPU Calculation ---
gpu_time = None # Initialize GPU time to None
option_price_gpu = None # Initialize GPU price to None
speedup = None # Initialize speedup to None

if run_gpu_calculation:
    dt = T / N_STEPS
    # Generate all random numbers needed on the CPU first
    # Shape is (N_SIMULATIONS, N_STEPS)
    # Use float64 matching numpy default and standard for financial calcs
    random_numbers_cpu = np.random.standard_normal((N_SIMULATIONS, N_STEPS)).astype(np.float64)

    # Determine kernel launch configuration
    threads_per_block = 512 # Common choice, can be tuned
    # Calculate the required number of blocks
    blocks_per_grid = (N_SIMULATIONS + threads_per_block - 1) // threads_per_block

    print(f"\nLaunching GPU kernel with {blocks_per_grid} blocks and {threads_per_block} threads per block ({blocks_per_grid * threads_per_block:,} total threads)...")

    # Allocate device memory and transfer data
    random_numbers_device = cuda.to_device(random_numbers_cpu)
    # Allocate space for payoffs on the device
    payoffs_device = cuda.device_array(N_SIMULATIONS, dtype=np.float64)

    start_time_gpu = time.time()

    # Launch the CUDA kernel
    # Pass the launch configuration (blocks_per_grid, threads_per_block) before arguments
    monte_carlo_option_kernel[blocks_per_grid, threads_per_block](
        S0, K, r, sigma, dt, N_STEPS, random_numbers_device, payoffs_device
    )

    # Synchronize device to ensure kernel completes before stopping timer
    cuda.synchronize()

    end_time_gpu = time.time()
    gpu_time = end_time_gpu - start_time_gpu

    # Transfer payoffs back to host (CPU)
    payoffs_cpu = payoffs_device.copy_to_host()

    # Calculate the final option price on the CPU (summation/average)
    total_payoff_gpu = np.sum(payoffs_cpu)
    option_price_gpu = (total_payoff_gpu / N_SIMULATIONS) * np.exp(-r * T)

    print(f"Option Price (GPU): {option_price_gpu:.4f}")
    print(f"Execution Time (GPU): {gpu_time:.4f} seconds")

    # --- Speedup Calculation ---
    speedup = cpu_time / gpu_time if gpu_time is not None and gpu_time > 0 else float('inf')
    print(f"Speedup (CPU vs. GPU): {speedup:.2f}x")

    print("----------------------------------------")
    print("Device used for calculations: GPU (via Numba CUDA kernel)")

else:
    # If no GPU was detected or available
    print("\nGPU calculation skipped because no CUDA capable GPU was detected or available.")
    print("----------------------------------------")


# --- Black-Scholes Comparison ---
black_scholes = black_scholes_price(S0, K, T, r, sigma)
print(f"Option Price (Black-Scholes): {black_scholes:.4f}")

print("----------------------------------------")


# --- Section to send results to Gemini LLM ---
print("\n--- Sending Results to Gemini LLM for Analysis ---")

# Configure the Gemini API - Get API key from environment variable
# Make sure you have set the GOOGLE_API_KEY environment variable
api_key = os.getenv("GEMINI_API_KEY")
if not api_key:
    print("Error: GEMINI_API_KEY environment variable not set.")
    print("Skipping LLM analysis.")
else:
    try:
        genai.configure(api_key=api_key)

        # Construct the prompt using collected results
        prompt_parts = [
            "Analyze the following performance and pricing results from a Monte Carlo European Call Option simulation:",
            "", # Add a blank line for readability in prompt
            f"Detected CUDA Capable GPUs: {gpu_count}",
            f"Number of Simulations: {N_SIMULATIONS:,}",
            f"Number of Time Steps: {N_STEPS}",
            f"Option Parameters: S0={S0}, K={K}, r={r}, sigma={sigma}, T={T}",
            "", # Blank line
            f"CPU Execution Time: {cpu_time:.4f} seconds",
            f"Monte Carlo Price (CPU): {option_price_cpu:.4f}",
            "", # Blank line
        ]

        if run_gpu_calculation:
             prompt_parts.extend([
                f"GPU Execution Time: {gpu_time:.4f} seconds",
                f"Monte Carlo Price (GPU): {option_price_gpu:.4f}",
                f"Speedup (CPU vs. GPU): {speedup:.2f}x",
                f"Device used for calculations: GPU (via Numba CUDA kernel)",
                "", # Blank line
            ])

        prompt_parts.extend([
            f"Black-Scholes Price (Benchmark): {black_scholes:.4f}",
            "", # Blank line
            "Please provide an analysis focusing on:",
            "1. The significance of the detected GPU count in the context of the run.",
            "2. The comparison between CPU and GPU execution times and the resulting speedup, commenting on the effectiveness of GPU acceleration.",
            "3. The agreement between the Monte Carlo results (CPU and GPU, if applicable) and the Black-Scholes benchmark, commenting on the simulation's accuracy.",
            "4. The practical implications of this performance difference for fields like quantitative finance or scientific computing, especially considering the availability of multiple GPUs.",
            "", # Blank line
            "Provide your analysis in a concise summary format.",
        ])

        # Join prompt parts into a single string
        prompt_text = "\n".join(prompt_parts)

        # For debugging, you might want to print the prompt before sending
        # print("--- PROMPT SENT TO GEMINI ---")
        # print(prompt_text)
        # print("-----------------------------")

        # Create the Generative Model instance
        # Use a suitable model name, e.g., 'gemini-pro', 'gemini-1.5-flash-latest', etc.
        #model = genai.GenerativeModel('YOUR_CHOSEN_MODEL_NAME_FROM_THE_LIST') # Replace this placeholder
        model = genai.GenerativeModel('models/gemini-2.0-flash') # Or another appropriate model

        # Generate content from the prompt
        # Use a timeout in case the API call hangs
        response = model.generate_content(prompt_text, request_options={'timeout': 60}) # 60 seconds timeout

        # Print the LLM's response
        print("\n--- Gemini LLM Analysis ---")
        # Check if the response has text content
        if response.candidates and response.candidates[0].content.parts:
             print(response.candidates[0].content.parts[0].text)
        else:
             print("Error: No text content received from LLM response.")
             if response.prompt_feedback:
                 print(f"Prompt Feedback: {response.prompt_feedback}")

        print("---------------------------")

    except Exception as e:
        print(f"\nError communicating with Gemini LLM API: {e}")
        print("Skipping LLM analysis.")

print("\nScript execution finished.")